[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timz815/360-NLP-Project/blob/main/finetune.ipynb)

In [2]:
# pip3 install unsloth
# pip3 install transformers datasets trl accelerate
# pip3 install torch torchvision torchaudio


In [4]:
from unsloth import FastLanguageModel

model_name = "unsloth/Qwen3-4B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    load_in_4bit=True,
    device_map="auto",
)

==((====))==  Unsloth 2025.11.4: Fast Qwen3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4060 Ti. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
import json
import torch

jsonl_path = r"C:\Users\timot\Downloads\nlp training\movie_dialogue.jsonl"
data = [json.loads(line) for line in open(jsonl_path, encoding="utf-8")]

In [6]:
def format_example(ex, tokenizer):
    messages = [
        {"role": "user",     "content": ex["english"]},
        {"role": "assistant", "content": ex["chinese"]},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return {"text": text}

In [7]:
def tokenize_example(ex, tokenizer, max_length=2048):
    tokens = tokenizer(ex["text"], truncation=True, max_length=max_length)
    return {"input_ids": tokens["input_ids"],
            "attention_mask": tokens["attention_mask"]}

ds = Dataset.from_list(data) \
            .map(format_example, fn_kwargs={"tokenizer": tokenizer},
                 remove_columns=data[0].keys(), num_proc=1) \
            .map(tokenize_example, fn_kwargs={"tokenizer": tokenizer},
                 remove_columns=["text"], num_proc=1)

Map (num_proc=1): 100%|███████████████████████████████████████████████████████| 161/161 [00:28<00:00,  5.74 examples/s]


In [8]:
from trl import SFTTrainer, SFTConfig
import torch

sft_config = SFTConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=20,
    max_steps=300,
    logging_steps=1,
    output_dir=r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora",
    optim="adamw_8bit",
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    completion_only_loss=False,
)

In [10]:
# === add LoRA adapters ===
model = FastLanguageModel.get_peft_model(
    model,
    r=16,               # rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing=True,  # saves memory
)

Unsloth 2025.11.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [11]:
import os, datasets
os.environ["TOKENIZERS_PARALLELISM"] = "false"
datasets.config.DEFAULT_MAX_BATCH_SIZE = 1

trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    tokenizer=tokenizer,
    args=sft_config,
    data_collator=None,
    formatting_func=None,
    dataset_text_field=None,
)

In [15]:
import torch
torch._dynamo.config.suppress_errors = True          # don't crash on compile fail
torch._dynamo.config.disable = True                  # disable dynamo completely


In [16]:
trainer.train()
model.save_pretrained(r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora\final")
tokenizer.save_pretrained(r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora\final")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 161 | Num Epochs = 15 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,5.450200
2,6.599200
3,6.262600
4,6.364500
5,4.863100
6,5.006900
7,5.088500
8,4.831200
9,3.979300
10,3.497900


('C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\tokenizer_config.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\special_tokens_map.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\chat_template.jinja',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\vocab.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\merges.txt',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\added_tokens.json',
 'C:\\Users\\timot\\Downloads\\nlp training\\qwen3-4b-dialogue-lora\\final\\tokenizer.json')

In [17]:
#QUICK TEST
FastLanguageModel.for_inference(model)

messages = [{"role": "user", "content": "I’m gonna make him an offer he can’t refuse."}]
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
out = model.generate(input_ids=inputs, max_new_tokens=128, temperature=0.7, do_sample=True)
print(tokenizer.decode(out[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


user
I’m gonna make him an offer he can’t refuse.
assistant
<think>

</think>

我要给他一个他无法拒绝的条件


In [ ]:
model.save_pretrained_gguf(
    r"C:\Users\timot\Downloads\nlp training\qwen3-4b-dialogue-lora/gguf",
    tokenizer,
    quantization_method="q4_k_m"
)

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: C:\Users\timot\.cache\huggingface\hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:  50%|████████████████████                    | 1/2 [01:40<01:40, 100.92s/it]